In [2]:
# result를 저장할 폴더 생성
# !mkdir results

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy  # Add Deepcopy for args
import seaborn as sns
import matplotlib.pyplot as plt

## Data Preparation

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)

# trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
# valloader = torch.utils.data.DataLoader(valset, batch_size=4, shuffle=False)
# testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

partition = {'train': trainset, 'val':valset, 'test':testset}

## Model Architecture

In [5]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        # in_channels : 채널 수 (RGB가 3차원 이니까 3)
        # out_channels : output tensor의 채널 개수, 그 layer의 filter개수와 같음, 보통 2의 x승으로 사용)
        # kernel_size : filter size와 동일, 3X3 filter -> 3
        # strid : filter를 몇 칸씩 이동 시킬 것인지, Default -> 1
        # padding : zero-padding을 얼마나 줄 것이지, Default -> 0. tensor의 H과 W의 사이즈 변동이 없으려면 2로 나눈 몫으로 설정)
        # bias : bias 항을 추가할 것인지, Default -> True       
        self.conv_layer1 = nn.Conv2d(in_channels=3,
                                     out_channels = 64,
                                     kernel_size = 3,
                                     stride = 1,
                                     padding = 1)
        self.conv_layer2 = nn.Conv2d(in_channels=64,
                                     out_channels = 256,
                                     kernel_size = 5,
                                     stride = 1,
                                     padding = 2) # dimension을 유지하기 위해서는 5를 2로 나눈 몫인 2로함? 유지하기를 원하면 kernel과 stride를 보고 계산해서 함
        self.acti = nn.ReLU()
        self.maxpool_layer1 = nn.MaxPool2d(kernel_size = 2,
                                     stride = 2) # strid 수는 계산해서 하는 건가? 내가 정하는 것!
        self.fc_layer = nn.Linear(65536, 10) # (input dimension, output dimension-> 10개의 확률값)
        
    def forward(self, x): # x.shape -> (2, 3, 32, 32)
        x = self.conv_layer1(x)   # x.shape -> (2, 64, 32, 32) 32가 유지되는 이유 : dimension을 유지하기 위해 padding을 1로 줬기 때문
        x = self.acti(x)   # x.shape -> (2, 64, 32, 32)
        x = self.conv_layer2(x) # x.shape -> (2, 256, 32, 32) 32가 유지되는 이유 : dimension을 유지하기 위해 padding을 2로 줬기 때문
        x = self.acti(x)  # x.shape -> (2, 256, 32, 32)
        x = self.maxpool_layer1(x) # x.shape -> (2, 256, 16, 16)
        x = x.view(x.size(0), -1) # fc에 넣기 위해 조정. batch_size는 그대로 유지하고, 나머지 값은 하나로 펴줌 # x.shape -> (2, 256*16*16)
        x = self.fc_layer(x) # x.shape -> (2, 10)
        return x                         


In [24]:
def dimension_check():
    net = CNN()
    x = torch.randn(2, 3, 32, 32) # torch.randn() : 명시한 차원에 따라서 랜덤 값을 만들어줌
    y = net(x)
    print(y.size())

In [25]:
# maxpool 안 했을 때
dimension_check() # 여기서 32, 32가 나오는 이유는 kernel size가 5라서..? 아님 그냥 바로 위 셀에서 32라고 지정해줘서?

torch.Size([2, 256, 32, 32])


In [27]:
# maxpool 넣었을 때
dimension_check() # depth 는 256, 사이즈만 16이 됐네?

torch.Size([2, 256, 16, 16])


In [29]:
# view 해줬을 때. 이게 linear layer의 input dimension이 됨
dimension_check()

torch.Size([2, 65536])


In [13]:
def train(net, partition, optimizer, criterion, args):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                              batch_size=args.train_batch_size,
                                              shuffle=True, num_workers=2)
    net.train() # train할 때만 적용되는 작업(dropout 등)을 수행 
    
   
    correct = 0
    total = 0
    train_loss = 0
    for i , data in enumerate(trainloader):
        optimizer.zero_grad() # tensor에는 tensor 자체의 값과 그 tensor의 gradient가 저장되어 있음. 새로 계산된 gradient가 변질되지 않도록 0으로 만들어 줌
        
        # get the inputs
        inputs, labels = data # inputs.shape -> (train_batch_size,3,32,32)
        # inputs = inputs.cuda()
        # labels = labels.cuda() # labels.shape -> (train_batch_size,1) 
        outputs = net(inputs) # outputs.shape -> (train_batch_size, 10)
        
        loss = criterion(outputs, labels) # batch 마다 각각 계산한 뒤 평균낸 값
        loss.backward() # Gradient 계산 : tensor에 저장되어 있는 gradient 값을 바꿔줌
        optimizer.step() # 저장되어있는 gradient 값을 기준으로 tensor 자체의 값을 바꿔줌
        
        train_loss += loss.item() # tensor 자체가 flow를 갖고 있어서 무거움. 그 안에 있는 숫자만 빼와서 누적함 
        _, predicted = torch.max(outputs.data, 1) # _ : max 값 / 안 쓰는 변수를 저장, predicted : index 값(몇 번째 index에 max가 있는지), predicted.shape -> (train_batch_size, 1(max의 index))
        total += labels.size(0) # labels.shape -> (train_batxh_size, 1), 결국 전체 몇 개의 데이터가 있는지 저장
        correct += (predicted == labels).sum().item() # True의 개수. tensor이기 때문에 item 함
   
    train_loss = train_loss / len(trainloader) # 전체 데이터들의 평균 loss 값
    train_acc = 100 * correct / total
    return net, train_loss, train_acc # net은 왜 return 하지? 안 해도됨

In [4]:
import torch
predicted = torch.tensor([1,2,3])
labels = torch.tensor([1,6,9])
(predicted == labels).sum()


tensor(1)

In [1]:
predicted = [1,2,3]
labels = [1,6,9]
predicted == labels

False

In [ ]:
def validate(net, partition, criterion, args):
    valloader = torch.utils.data.DataLoader(partition['val'],
                                            batch_size=args.test_batch_size,
                                            shuffle=False, num_workers=2)
    
    net.eval()
    
    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad(): # gradient를 아예 계산하지 않도록. flow를 계산하지 않도록. -> 속도랑 메모리 향상
        for i, data in enumerate(valloader):
            images, labels = data
            # images = images.cuda()
            # labels = labels.cuda()
            outputs = net(images)
            
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(val_loss.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [ ]:
def test(net, partition, args):
    testloader = torch.utils.data.DataLoader(testset, batch_size=args.test_batch_size,
                                             shuffle=False, num_workers=2)
    
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad:
        for data in testloader:
            images, labels = data
            # images = images.cuda()
            # labels = labels.cuda()
            
            outputs = net(images)
            _, predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            correct += (predicted, labels).sum().item()
            
        test_acc = 100 * correct / total
    return test_acc

In [ ]:
def experiment(partition, args):
    
    net = CNN()
    # net.cuda()
    
    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2) # weight_decay가 l2 값을 얼마나 적용할지었나?
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice') # try except 구문도 많이 사용하나? 
    
    train_losses_lst = []
    val_losses_lst = []
    train_accs_lst = []
    val_accs_lst = []
    
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time() # time.perf_counter()를 권장 -> cpu 시간이 얼마나 걸렸는지 체크. 이 코드가 얼마나 걸렸는지 알려줌. time.time()보다 좀 더 정확 : 현재 시간
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()
        
        train_losses_lst.append(train_loss)  # 얘네가 들어가는 과정...?
        val_losses_lst.append(val_loss)
        train_accs_lst.append(train_acc)
        val_accs_lst.append(val_acc)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took')
        
    test_acc = test(net, partition, args)
    
    result = {}
    result['train_losses'] = train_losses_lst
    result['val_losses'] = val_losses_lst
    result['train_accs'] = train_accs_lst
    result['val_accs'] = val_accs_lst
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result # vars(args)는 뭐지? args 타입을 dict로 바꿔주는 것. args를 print 하거나, 저장하기 위함
        

In [ ]:
# =====Random Seed Initialization===== #
seed = 123
np.random.seed(seed) # np의 random 함수에 대한 seed
torch.manual_seed(seed) # torch의 random 함수에 대한 seed

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = 'exp1_n_layer_hid_dim'

# =====Model Capcity===== #
args.out_dim = 10

args.act = 'relu'

# ====Regularization==== #
args.l2 = 0.00001
# args.use_xavier = True # 파라미터 초기값의 방법 중 하나. 많이 쓰진 않음. 보통 torch가 default값을 알아서 적용하도록 함

# =====Optimizer & Training===== #
args.optim = 'RMSprop'
args.lr = 0.0015
args.epoch = 10

args.train_batch_size = 256
args.test_batch_size = 1024

# =====Experiment Variable===== #
name_var1 = 'lr'
name_var2 = 'l2'
list_var1 = [0.0001]
list_var2 = [0.00001]

for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1) # setattr이 뭐지
        setattr(args, name_var2, var2)
        print(args)
        
        setting, result = experiment(partition, deepcopy(args))
        save_exp_result(setting, result)



